In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
df_disc_data = pd.read_csv('missing_values_discovery_time_dataset.csv')
df_cont_data = pd.read_csv('dataset_filled_contdate_time.csv')
df_original = pd.read_csv('sampled_dataset_reduced.csv')

In [2]:
df_original.drop(columns='FIRE_NAME',inplace=True) # removing columns since it has data mismatch and no help to estimate the values for this col

In [3]:
df_original.isna().sum()

FOD_ID                             0
NWCG_REPORTING_AGENCY              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_TIME                203216
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                     210814
CONT_TIME                     226188
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_DESCR                        0
STATE                              0
COUNTY                        245629
dtype: int64

### Update Discovery Time

In [4]:
df_original['DISCOVERY_TIME_1'] = df_disc_data['DISCOVERY_TIME']
df_original.drop(columns=['DISCOVERY_TIME'],inplace=True)
df_original.rename(columns={'DISCOVERY_TIME_1':'DISCOVERY_TIME',},inplace=True)
df_original.shape

(593378, 17)

### Updating Cont Date and Time 

In [5]:
df_original['CONT_DATE_1'] = df_cont_data['CONT_DATE']
df_original['CONT_TIME_1'] = df_cont_data['CONT_TIME']
df_original.drop(columns=['CONT_DATE','CONT_TIME'],inplace=True)
df_original.rename(columns={'CONT_DATE_1':'CONT_DATE','CONT_TIME_1':'CONT_TIME'},inplace=True)
df_original.shape

(593378, 17)

In [6]:
df_original.to_csv('preprocessed_dataset.csv',index=False)

### Sampling


In [7]:
df_fire = pd.read_csv('preprocessed_dataset.csv')
df_fire

FOD_ID NWCG_REPORTING_AGENCY SOURCE_REPORTING_UNIT_NAME  FIRE_YEAR  \
0               2                    FS   Eldorado National Forest       2004   
1               4                    FS   Eldorado National Forest       2004   
2               5                    FS   Eldorado National Forest       2004   
3               6                    FS   Eldorado National Forest       2004   
4               7                    FS   Eldorado National Forest       2004   
...           ...                   ...                        ...        ...   
593373  300348260                ST/C&L  CDF - San Bernardino Unit       2015   
593374  300348276                ST/C&L          Lassen-Modoc Unit       2015   
593375  300348290                ST/C&L           Santa Clara Unit       2015   
593376  300348354                ST/C&L        Shasta-Trinity Unit       2015   
593377  300348361                ST/C&L    Humboldt-Del Norte Unit       2015   

       DISCOVERY_DATE  STAT_CAUSE_CODE STAT_CAUSE_DESCR  FIRE_SIZE  \
0          2004-05-12              1.0        Lightning       0.25   
1          2004-06-28              1.0        Lightning       0.10   
2          2004-06-28              1.0        Lightning       0.10   
3          2004-06-30              1.0        Lightning       0.10   
4          2004-07-01              1.0        Lightning       0.10   
...               ...              ...              ...        ...   
593373     2015-07-25              7.0            Arson       0.50   
593374     2015-06-26              1.0        Lightning       0.05   
593375     2015-06-27              7.0            Arson       0.10   
593376     2015-09-30              7.0            Arson       1.00   
593377     2015-08-01              1.0        Lightning       4.00   

       FIRE_SIZE_CLASS   LATITUDE   LONGITUDE        OWNER_DESCR STATE COUNTY  \
0                    A  38.933056 -120.404444               USFS    CA     61   
1                    A  38.559167 -119.913333               USFS    CA      3   
2                    A  38.559167 -119.933056               USFS    CA      3   
3                    A  38.635278 -120.103611               USFS    CA      5   
4                    A  38.688333 -120.153333               USFS    CA     17   
...                ...        ...         ...                ...   ...    ...   
593373               B  34.244444 -116.658889  UNDEFINED FEDERAL    CA    NaN   
593374               A  40.443000 -120.664433   STATE OR PRIVATE    CA    NaN   
593375               A  38.041150 -121.985283  UNDEFINED FEDERAL    CA    NaN   
593376               B  40.588583 -123.069617   STATE OR PRIVATE    CA    NaN   
593377               B  40.244833 -123.544167  UNDEFINED FEDERAL    CA    NaN   

        DISCOVERY_TIME   CONT_DATE  CONT_TIME  
0                  845  2004-05-12     1530.0  
1                 1600  2004-07-03     1400.0  
2                 1600  2004-07-03     1200.0  
3                 1800  2004-07-01     1600.0  
4                 1800  2004-07-02     1400.0  
...                ...         ...        ...  
593373            1943  2015-07-25     2050.0  
593374            1752  2015-06-26     2014.0  
593375             337  2015-06-27      242.0  
593376            2357  2015-10-01       56.0  
593377            1331  2015-08-06     1000.0  

[593378 rows x 17 columns]

In [8]:
from sklearn.model_selection import train_test_split
X = df_fire.drop(columns='STAT_CAUSE_CODE')
y = df_fire['STAT_CAUSE_CODE']

In [9]:
X_inter,X_test,y_inter,y_test = train_test_split(X,y,shuffle=True,random_state=15,test_size=0.2)
X_train,X_valid,y_train,y_valid = train_test_split(X_inter,y_inter, shuffle=False,random_state=2018,test_size=0.25)
del X_inter, y_inter
print("train: {}% | validation: {}% | test {}%".format(round(len(y_train)/len(y),2),
                                                       round(len(y_valid)/len(y),2),
                                                       round(len(y_test)/len(y),2)))

train: 0.6% | validation: 0.2% | test 0.2%


In [10]:
train_set = X_train
train_set['STAT_CAUSE_CODE'] = y_train
train_set.to_csv('training_dataset.csv', index=False)
train_set

FOD_ID NWCG_REPORTING_AGENCY  \
308391     908711                ST/C&L   
561510  300002788                    FS   
457904    1755920                ST/C&L   
26656       43168                    FS   
190337     347309                   BLM   
...           ...                   ...   
339910     995809                ST/C&L   
302435     870227                ST/C&L   
450717    1650948                ST/C&L   
545143  201763898                    FS   
189636     346158                   BLM   

                        SOURCE_REPORTING_UNIT_NAME  FIRE_YEAR DISCOVERY_DATE  \
308391          KYS Southeastern Kentucky District       1999     1999-09-10   
561510                        Gila National Forest       2014     2014-07-14   
457904  Washington Department of Natural Resources       2009     2009-07-31   
26656                         Lolo National Forest       1992     1992-08-19   
190337                     Farmington Field Office       2005     2005-06-03   
...                                            ...        ...            ...   
339910                                 GAS Unit 32       1998     1998-06-30   
302435               Washington State Headquarters       1994     1994-07-13   
450717                 Georgia Forestry Commission       2000     2000-08-13   
545143                Wallowa-Whitman National For       2013     2013-07-17   
189636                  Winnemucca District Office       2007     2007-07-16   

       STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE   LONGITUDE  \
308391            Arson        5.0               B  37.116100  -85.188100   
561510        Lightning        0.1               A  33.150000 -108.000000   
457904        Lightning       60.0               C  46.820430 -120.801200   
26656         Lightning        0.5               B  47.075000 -114.265000   
190337        Lightning        0.1               A  36.850800 -107.522600   
...                 ...        ...             ...        ...         ...   
339910        Lightning        1.1               B  31.378100  -81.656400   
302435        Lightning        0.8               B  46.555860 -121.185560   
450717        Lightning        4.5               B  32.303800  -82.519100   
545143        Lightning        0.3               B  45.411389 -118.136944   
189636        Lightning       44.0               C  40.157400 -117.226800   

                  OWNER_DESCR STATE         COUNTY  DISCOVERY_TIME  \
308391  MISSING/NOT SPECIFIED    KY            NaN            1498   
561510                   USFS    NM          Grant            1536   
457904                  STATE    WA  Yakima County            1200   
26656                    USFS    MT            NaN            1705   
190337                  STATE    NM     Rio Arriba            1432   
...                       ...   ...            ...             ...   
339910  MISSING/NOT SPECIFIED    GA            NaN            1578   
302435                PRIVATE    WA  Yakima County            1569   
450717                PRIVATE    GA     Montgomery            1415   
545143                   USFS    OR            061            1550   
189636                    BLM    NV       Humboldt            2105   

         CONT_DATE  CONT_TIME  STAT_CAUSE_CODE  
308391  1999-09-10     2300.0              7.0  
561510  2014-07-14     1630.0              1.0  
457904  2009-09-10     1200.0              1.0  
26656   1992-08-20      800.0              1.0  
190337  2005-06-03     1630.0              1.0  
...            ...        ...              ...  
339910  1998-07-10     1315.0              1.0  
302435  1994-07-13     1035.0              1.0  
450717  2000-08-13     1415.0              1.0  
545143  2013-07-18     1028.0              1.0  
189636  2007-07-17     1400.0              1.0  

[356026 rows x 17 columns]

In [11]:
valid_set = X_valid
valid_set['STAT_CAUSE_CODE'] = y_valid
valid_set.to_csv('validation_dataset.csv', index=False)
valid_set

FOD_ID NWCG_REPORTING_AGENCY  \
10307       15880                    FS   
399550    1176028                ST/C&L   
201198     362780                   NPS   
49838       78762                    FS   
187499     343358                   BLM   
...           ...                   ...   
554699  201830698                ST/C&L   
44231       69504                    FS   
35483       56838                    FS   
199301     359573                   BLM   
446197    1619745                ST/C&L   

                               SOURCE_REPORTING_UNIT_NAME  FIRE_YEAR  \
10307                             Payette National Forest       2006   
399550                            NCS Region 3 District 9       2001   
201198                               Arches National Park       2009   
49838                                Gila National Forest       1995   
187499                                Ely District Office       2002   
...                                                   ...        ...   
554699  New Jersey Department of Environmental Protection       2013   
44231                            Sawtooth National Forest       1994   
35483                               Dixie National Forest       1993   
199301                          Rock Springs Field Office       2003   
446197                         Oklahoma Forestry Services       2009   

       DISCOVERY_DATE STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE  \
10307      2006-08-07        Lightning        1.0               B  45.024167   
399550     2001-12-30            Arson       40.0               C  35.050000   
201198     2009-08-06        Lightning        0.1               A  38.775310   
49838      1995-07-17        Lightning        0.1               A  33.871667   
187499     2002-07-12        Lightning        0.3               B  37.450200   
...               ...              ...        ...             ...        ...   
554699     2013-04-10            Arson        5.0               B  39.553700   
44231      1994-08-02        Lightning        0.1               A  44.166667   
35483      1993-08-03        Lightning        0.1               A  37.666667   
199301     2003-06-19        Lightning        0.1               A  41.755500   
446197     2009-02-22            Arson       15.0               C  35.708900   

         LONGITUDE            OWNER_DESCR STATE      COUNTY  DISCOVERY_TIME  \
10307  -115.538611                   USFS    ID          85            1951   
399550  -84.066700  MISSING/NOT SPECIFIED    NC         NaN            1536   
201198 -109.593920                    NPS    UT         NaN            1822   
49838  -108.480000                   USFS    NM         NaN            1430   
187499 -114.999700                    BLM    NV     Lincoln            1624   
...            ...                    ...   ...         ...             ...   
554699  -74.351300                  STATE    NJ  Burlington            1800   
44231  -114.663611                   USFS    ID         NaN            1615   
35483  -112.300000                   USFS    UT         NaN            1700   
199301 -109.572100                    BLM    WY  Sweetwater            1430   
446197  -94.764183                PRIVATE    OK       Adair            1420   

         CONT_DATE  CONT_TIME  STAT_CAUSE_CODE  
10307   2006-09-12     1640.0              1.0  
399550  2001-12-30     1915.0              7.0  
201198  2009-08-06     2015.0              1.0  
49838   1995-07-17     1800.0              1.0  
187499  2002-07-13      800.0              1.0  
...            ...        ...              ...  
554699  2013-04-23     1400.0              7.0  
44231   1994-08-02     1800.0              1.0  
35483   1993-08-03     2100.0              1.0  
199301  2003-06-19     1700.0              1.0  
446197  2009-02-22     1420.0              7.0  

[118676 rows x 17 columns]

In [12]:
test_set = X_test
test_set['STAT_CAUSE_CODE'] = y_test
test_set.to_csv('test_dataset.csv', index=False)
test_set

FOD_ID NWCG_REPORTING_AGENCY  \
300374   861011                ST/C&L   
105389   164947                    FS   
97516    153471                    FS   
353021  1051346                ST/C&L   
460618  1772216                ST/C&L   
...         ...                   ...   
142370   241759                   BLM   
245519   545783                ST/C&L   
321524   953966                ST/C&L   
40431     64472                    FS   
162977   294010                   BIA   

                       SOURCE_REPORTING_UNIT_NAME  FIRE_YEAR DISCOVERY_DATE  \
300374                              John Day Unit       2001     2001-08-15   
105389                  Dakota Prairie Grasslands       2003     2003-07-08   
97516              San Bernardino National Forest       2002     2002-10-17   
353021                                 SCS Unit 4       2001     2001-03-14   
460618                Fire Department of New York       2007     2007-06-29   
...                                           ...        ...            ...   
142370                        Vernal Field Office       1999     1999-06-30   
245519  Minnesota Department of Natural Resources       2002     2002-04-20   
321524                     MSS Southeast District       2001     2001-03-18   
40431                 Black Hills National Forest       1994     1994-06-04   
162977                   Northern Cheyenne Agency       2004     2004-05-07   

       STAT_CAUSE_DESCR  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE   LONGITUDE  \
300374        Lightning        0.2               A  45.029910 -119.603970   
105389        Lightning        3.0               B  46.659444 -103.499444   
97516         Lightning        0.1               A  34.283611 -116.887778   
353021            Arson        0.5               B  33.883300  -79.083300   
460618            Arson        0.1               A  43.325486  -77.934956   
...                 ...        ...             ...        ...         ...   
142370        Lightning        1.5               B  39.851900 -110.417900   
245519            Arson       30.0               C  48.802719  -95.895780   
321524            Arson        1.0               B  30.748900  -89.550800   
40431         Lightning        0.1               A  43.818333 -103.966667   
162977        Lightning        0.1               A  45.565500 -106.634200   

                  OWNER_DESCR STATE  COUNTY  DISCOVERY_TIME   CONT_DATE  \
300374                PRIVATE    OR      49            1195  2001-08-15   
105389                   USFS    ND       7            1630  2003-07-08   
97516                    USFS    CA     NaN            1503  2002-10-17   
353021  MISSING/NOT SPECIFIED    SC     NaN            1510  2001-03-14   
460618  MISSING/NOT SPECIFIED    NY  MONROE            1435  2007-06-29   
...                       ...   ...     ...             ...         ...   
142370                    BLM    UT  Uintah            1945  1999-07-02   
245519  MISSING/NOT SPECIFIED    MN  Roseau            1491  2002-04-21   
321524  MISSING/NOT SPECIFIED    MS     NaN            1655  2001-03-18   
40431                    USFS    SD     NaN            1840  1994-06-05   
162977                    BIA    MT     NaN            2005  2004-05-07   

        CONT_TIME  STAT_CAUSE_CODE  
300374     2230.0              1.0  
105389     1720.0              1.0  
97516      1548.0              1.0  
353021     1400.0              7.0  
460618     1435.0              7.0  
...           ...              ...  
142370     1200.0              1.0  
245519     1800.0              7.0  
321524     1750.0              7.0  
40431       200.0              1.0  
162977     2202.0              1.0  

[118676 rows x 17 columns]